<a href="https://www.kaggle.com/code/joshuaokolo/predicting-molecular-properties?scriptVersionId=105000621" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Predicting Molecular Properties based on the Stanford DeepChem Library

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('max_rows', 5)
import os
import matplotlib.pylab as plt
import seaborn as sns
from sklearn import metrics
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
plt.style.use('ggplot')
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]
import os
from IPython.display import Image, HTML, display

In [ ]:
!pip install deepchem

In [ ]:
%%bash -e
if ! [[ -f ./xyz2mol.py ]]; then
  wget https://raw.githubusercontent.com/jensengroup/xyz2mol/master/xyz2mol.py
fi

In [ ]:
pip install py3Dmol # This is built on the object-oriented, webGL based JavaScript library for online molecular visualization 3Dmol.js (Rego & Koes, 2015);

In [ ]:
file_folder = '../input/champs-scalar-coupling' if 'champs-scalar-coupling' in os.listdir('../input/') else '../input'

In [ ]:
train = pd.read_csv(f'{file_folder}/train.csv')[0-60000:]

In [ ]:
train.head()

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} -c rdkit rdkit

In [ ]:
# Few Snippets from https://www.kaggle.com/sunhwan/using-rdkit-for-atomic-feature-and-visualization
# rdkit & xyz2mol
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules
from rdkit.Chem import Draw
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions #Only needed if modifying defaults
DrawingOptions.bondLineWidth=1.8
from rdkit.Chem.rdmolops import SanitizeFlags

# https://github.com/jensengroup/xyz2mol
from xyz2mol import xyz2mol, xyz2AC, AC2mol, read_xyz_file
from pathlib import Path
import pickle

CACHEDIR = Path('./')

def chiral_stereo_check(mol):
    # avoid sanitization error e.g., dsgdb9nsd_037900.xyz
    Chem.SanitizeMol(mol, SanitizeFlags.SANITIZE_ALL - SanitizeFlags.SANITIZE_PROPERTIES)
    Chem.DetectBondStereochemistry(mol,-1)
    # ignore stereochemistry for now
    #Chem.AssignStereochemistry(mol, flagPossibleStereoCenters=True, force=True)
    #Chem.AssignAtomChiralTagsFromStructure(mol,-1)
    return mol

def xyz2mol(atomicNumList,charge,xyz_coordinates,charged_fragments,quick):
    AC,mol = xyz2AC(atomicNumList,xyz_coordinates)
    new_mol = AC2mol(mol,AC,atomicNumList,charge,charged_fragments,quick)
    new_mol = chiral_stereo_check(new_mol)
    return new_mol

def MolFromXYZ(filename):
    mol=''
    charged_fragments = True
    quick = True
    cache_filename = CACHEDIR/f'{filename.stem}.pkl'
    if cache_filename.exists():
        return pickle.load(open(cache_filename, 'rb'))
    else:
        try:
            atomicNumList, charge, xyz_coordinates = read_xyz_file(filename)
            mol = xyz2mol(atomicNumList, charge, xyz_coordinates, charged_fragments, quick)
            # commenting this out for kernel to work.
            # for some reason kernel runs okay interactively, but fails when it is committed.
            pickle.dump(mol, open(cache_filename, 'wb'))
        except:
            print(filename)
    return mol
#mol = MolFromXYZ(xyzfiles[1])
#m = Chem.MolFromSmiles(Chem.MolToSmiles(mol, allHsExplicit=True)); m

from multiprocessing import Pool
from tqdm import *
from glob import glob

def MolFromXYZ_(filename):
    return filename.stem, MolFromXYZ(filename)

mols = {}
n_cpu = 4
with Pool(n_cpu) as p:
    molecule_names = np.concatenate([train.molecule_name.unique()])
    molecule_names = molecule_names[:400]
    xyzfiles = [Path(f'../input/structures/{f}.xyz') for f in molecule_names]
    n = len(xyzfiles)
    with tqdm(total=n) as pbar:
        for res in p.imap_unordered(MolFromXYZ_, xyzfiles):
            mols[res[0]] = res[1]
            pbar.update()

In [ ]:
molecule_names = np.concatenate([train.molecule_name.unique()])
xyzfiles = [(f'{f}.xyz') for f in molecule_names]
few_molecule_names = molecule_names[:5]
few_molecule_names

In [ ]:
# Create mols format from xyz files
# Create smiles format

try:
    for molecule_name in few_molecule_names:
        #print('Molecule: {}'.format(molecule_name))
        m = MolFromXYZ(Path(f'../input/champs-scalar-coupling/structures/{molecule_name}.xyz'))
        smile_fromMolecule = Chem.MolToSmiles(m)
        print('Smile format from molecule: {}'.format(smile_fromMolecule))

except:
    print ('...something be goofy!!')
    pass 

In [ ]:
# Inspired by https://github.com/greglandrum/rdkit_blog/tree/master/notebooks

import py3Dmol # Amazing library for 3D visualization
from rdkit import Chem
from rdkit.Chem import AllChem
from ipywidgets import interact, interactive, fixed
def drawit(m,p,confId=-1):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p.removeAllModels()
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p.show()
p = py3Dmol.view(width=400,height=400)

# now construct the 3d view:
m = MolFromXYZ(Path(f'../input/champs-scalar-coupling/structures/dsgdb9nsd_129764.xyz'))
smile_fromMolecule = Chem.MolToSmiles(m)
m = Chem.MolFromSmiles(smile_fromMolecule)
#print(smile_fromMolecule)
m = Chem.AddHs(m)
AllChem.EmbedMultipleConfs(m,randomSeed=0xf00d,useExpTorsionAnglePrefs=True, useBasicKnowledge=True)
interact(drawit, m=fixed(m),p=fixed(p));

''' Trick: If you want click and drag on the image to spin the molecule, and scroll for zoom!!'''

In [ ]:
potential_energyDF = pd.read_csv(f'{file_folder}/potential_energy.csv')

In [ ]:
potential_energyDF.head()

In [ ]:
# Plot the distribution of potential_energy
potential_energyDF['potential_energy'].plot(kind='hist', figsize=(25, 5), bins=500, title='Distribution of Molecular Potential Energy', color='g')
plt.show()

In [ ]:
import tensorflow as tf
import deepchem as dc
import numpy as np
import pandas as pd

# Getting only few samples
molecule_names = molecule_names[0:400]

In [ ]:
# Create smiles format
xyzfiles = [(f'{f}.xyz') for f in molecule_names]
n = len(xyzfiles)
df_temp = pd.DataFrame({'molecule_name':[],'smile_fromMolecule': []})
try:
    for molecule_name in molecule_names:
        #print('Molecule: {}'.format(molecule_name))
        m = MolFromXYZ(Path(f'../input/champs-scalar-coupling/structures/{molecule_name}.xyz'))
        smile_fromMolecule = Chem.MolToSmiles(m)
        #print('Smile format from molecule: {}'.format(smile_fromMolecule))
        df_temp = df_temp.append({'molecule_name': molecule_name,'smile_fromMolecule': smile_fromMolecule}, ignore_index=True)
except:
    print ('...something is wrong!!')
    pass 

In [ ]:
df_temp.head()

In [ ]:
# Merge two dataframes
potential_energyDF_Smiles = pd.merge(df_temp, potential_energyDF, how = 'left', on='molecule_name')
potential_energyDF_Smiles.dropna(inplace=True)

In [ ]:
potential_energyDF_Smiles.shape

In [ ]:
# Our dataset file must contain a column with the SMILES sentence and another with our target (potential energy) in order to use with DeepChem
potential_energyDF_Smiles[['potential_energy', 'smile_fromMolecule']].to_csv('../potential_energyDF_Smiles.csv', index=False)

In [ ]:
from rdkit import Chem
import random
from deepchem.feat import CircularFingerprint
import deepchem as dc
import numpy as np

In [ ]:
# Dataset file
dataset_file = '../potential_energyDF_Smiles.csv'

In [ ]:
# Featurizer will create our fingerprint, and turn it into an array with 1024 bits.
featurizer = dc.feat.CircularFingerprint(size=1024)

# Prepare our dataset file
loader = dc.data.CSVLoader(
      tasks=["potential_energy"], smiles_field="smile_fromMolecule",
      featurizer=featurizer)

dataset = loader.featurize(dataset_file)

# Split train, validation and test
splitter = dc.splits.ScaffoldSplitter(dataset_file)
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)
train_mols = [Chem.MolFromSmiles(compound)
              for compound in train_dataset.ids]
valid_mols = [Chem.MolFromSmiles(compound)
              for compound in valid_dataset.ids]
# Normalize them
transformers = [dc.trans.NormalizationTransformer(transform_y=True, dataset=train_dataset)]

for dataset in [train_dataset, valid_dataset, test_dataset]:
    for transformer in transformers:
        dataset = transformer.transform(dataset)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Our regressor
sklearn_model = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=20, 
                                      min_samples_split=2, min_samples_leaf=1, 
                                      min_weight_fraction_leaf=0.0, max_features=None,
                                      max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                      min_impurity_split=None, bootstrap=True, oob_score=False, 
                                      n_jobs=None, random_state=1, warm_start=False)
model = dc.models.SklearnModel(sklearn_model)
model.fit(train_dataset)

# Evaluation
from deepchem.utils.evaluate import Evaluator
metric = dc.metrics.Metric(dc.metrics.r2_score)
evaluator = Evaluator(model, valid_dataset, transformers)
r2score = evaluator.compute_model_performance([metric])
print(r2score)

In [ ]:
# let’s plot the predicted R2 scores versus the true R2 scores for the constructed model.
llim = -600
ulim = -300
task = "measured potential_energy"
predicted_test = model.predict(test_dataset)
true_test = test_dataset.y
plt.scatter(predicted_test, true_test)
plt.xlim((llim, ulim))
plt.ylim((llim, ulim))
plt.plot([llim, ulim], [llim, ulim])
plt.xlabel('Predicted potential_energy')
plt.ylabel('True potential_energy')
plt.title(r'RF- predicted vs. true potential_energy')
plt.show()